In [1]:
from pathlib import Path
import uuid
import sys 
import os 

from featrixclient import Featrix
from featrixclient.featrix_embedding_space import FeatrixEmbeddingSpace
from featrixclient.featrix_project import FeatrixProject
from featrixclient.featrix_neural_function import FeatrixNeuralFunction
from featrixclient.featrix_upload import FeatrixUpload
from featrixclient.featrix_job import FeatrixJob
from featrixclient.utils import featrix_wrap_pd_read_csv

# be sure you have your client id and client secret in your environment... 
# 
# os.environ['FEATRIX_CLIENT_ID'] = "your client id"
# os.environ['FEATRIX_CLIENT_SECRET'] = "your client secret"
os.environ['FEATRIX_CLIENT_ID'] = "65f9fc5c94e2d92af20ff47b"
os.environ['FEATRIX_CLIENT_SECRET'] = "mtr_t4of2fHmlOs8tD0ABJi/0CNPCTeqYzSdx+v2zX/nEEQ="
url = "https://stage.featrix.com"
# url = "https://app.featrix.com"
# url = "http://localhost:3001"

fc = Featrix(url=url)

projects = fc.projects()


/Users/mtr/.python/fc_os/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# List the projects we have in our account
for p in projects:
    print(f"Project: {p.name} ({p.id} -- associated data {len(p.associated_uploads)})")

Project: DemoProject (66b50198ed34d7f6b8a12e21 -- associated data 1)
Project: Testing FC dece9198-f1e4-4272-858c-e05173637940 (66b3d350fd34ec26b6c74649 -- associated data 1)
Project: Testing FC 2db2d2c2-7540-44a9-a3d3-665b80f4fa46 (66b3d08a3f9e71e869407aba -- associated data 1)
Project: Testing FC 383f3a65-d7bb-4ffc-8da5-5afce0181887 (66b3ba666b2d29b4e5b80b5a -- associated data 1)
Project: Testing FC 67488b81-2c94-4176-b3e8-3e6a52eed356 (66b3b75cfcba35085277c8fc -- associated data 1)
Project: Testing FC (66b3b30dfcba35085277c8f9 -- associated data 1)
Project: jul11-two (668fe64a9ed407f5cb6fe419 -- associated data 1)
Project: jul9final (668dcee4d69f6f60bee3e141 -- associated data 1)
Project: ddd (668d7a1c9cdacf6386bb3581 -- associated data 1)
Project: flex (668009b321b9c3d2121087b4 -- associated data 1)


In [2]:
fc.create_project("DemoProject")

FeatrixException: Error with request: Project DemoProject already exists

In [ ]:
#
#  Create a new project and train an embedding space in it 
#

project = fc.create_project(f"Testing FC {uuid.uuid4()}")
upload = fc.upload_file(Path("../data/animals-1k.csv"))
# If you already have it uploaded, you can find it with:
# upload = fc.get_upload(filename="animals-1k.csv")
project = project.associate(upload)
if project.ready() is False:
    jobs = upload.get_jobs()
    jobs[-1].wait_for_completion("Processing Upload...")
    
es = fc.create_embedding_space(project, "Testing Embedding Space")
jobs = es.get_jobs()
# Should be 1 here -- the training job (there is also a create projection, but that isnt' a training job...
assert(1 == len(jobs))
assert(2 == len(es.get_jobs(active=True, training=False)))
training = jobs[0]
print(f"Job {training.job_type} ({training.id}) started")
training.wait_for_completion("Training...")


Training... No status yet


In [5]:
#
#  Encode a record  
records = es.encode_record(Path("../data/animals-1kv2.csv"))
# print(records)

# df = featrix_wrap_pd_read_csv(Path("../data/animals-1kv2.csv"))
# records2 = es.encode_record(df)
# # print(records2)

	TRACE CSV: working with file_path = ../data/animals-1kv2.csv


FeatrixConnectionError: Connection error for URL https://stage.featrix.com/api/neural/embedding_space/fast-encode-records: __No more retries, multiple errors __

In [4]:
es = FeatrixEmbeddingSpace.by_id('66b3d351fd34ec26b6c7464c', fc)

In [6]:
es.delete()

FeatrixConnectionError: Connection error for URL https://stage.featrix.com/api/neural/embedding_space/66b3d351fd34ec26b6c7464c: __No more retries, multiple errors __

In [3]:
#
# Create a neural function in a new project
#

project = fc.create_project(f"Testing FC {uuid.uuid4()}")
upload = fc.upload_file(Path("../data/animals-1kv3.csv"))
# If you already have it uploaded, you can find it with:
# upload = fc.get_upload(filename="animals-1k.csv")
project = project.associate(upload)
if project.ready() is False:
    jobs = upload.get_jobs()
    jobs[-1].wait_for_completion("Processing Upload...")
    
nf = fc.create_neural_function(target_fields="Height", project=project)
es = FeatrixEmbeddingSpace.by_id(nf.embedding_space_id, fc)
es_jobs = es.get_jobs()
nf_jobs = nf.get_jobs()
assert(1 == len(es_jobs))
assert(1 == len(nf_jobs))
# wait for the es
es_jobs[0].wait_for_completion("Embedding Space Training...")
nf_jobs[0].wait_for_completion("Neural Function Training...")

Called project.ready(False
Called project.ready(False
Initially the not ready count is 0
ES jobs 1:


In [3]:
project = projects[0]
embedding_list = project.embedding_spaces()
print(f"project {project.id} embeddings: {embedding_list[0].project_id} {len(embedding_list)}")
nf_list = project.neural_functions(embedding_space=embedding_list[0])

project 66b3ba666b2d29b4e5b80b5a embeddings: 66b3ba666b2d29b4e5b80b5a 1


In [6]:
es = es.refresh()
es.create_neural_function(target_fields="weight", wait_For_completion=True)

Called project.ready(False
Initially the not ready count is 0


FeatrixNeuralFunction(id='66b3d6333f9e71e869407ac1', created_by=ObjectId('65f9fc5c94e2d92af20ff47b'), created_at=datetime.datetime(2024, 8, 7, 20, 16, 51, 625000), updated_at=datetime.datetime(2024, 8, 7, 20, 16, 51, 626000), name='predict_weight', organization_id=ObjectId('65f9fc5a94e2d92af20ff47a'), project_id=ObjectId('66b3d350fd34ec26b6c74649'), embedding_space_id=ObjectId('66b3d351fd34ec26b6c7464c'), associated_uploads=[UploadAssociation(upload_id=ObjectId('66b3d08a3f9e71e869407abc'), label='animals-1k', sample_percentage=1.0, sample_row_count=0, drop_duplicates=False, upload=None)], target_columns=['weight'], training_rows=0, training_input_columns=[], training_num_uniques=None, training_num_not_nulls=None, training_target_histogram=None, training_metrics=None, learning_rate=0.0001, epochs=0, model_size='small', target_type=None, mlp_predictor=None, job_id=None, training_state=<TrainingState.PENDING: 'training-pending'>, training_credits_budgeted=3.0, training_credits_actual=0.0,

In [20]:
jobs = project.jobs()
print(jobs[0].model_dump_json(indent=2))

{
  "id": "66b3b4916b2d29b4e5b80b59",
  "created_by": "65f9fc5c94e2d92af20ff47b",
  "created_at": "2024-08-07T17:53:21.010000",
  "updated_at": "2024-08-07T17:53:21.203000",
  "organization_id": "65f9fc5a94e2d92af20ff47a",
  "job_type": "embedding-space-projection-create",
  "readonly_object": false,
  "project_id": "66b3b30dfcba35085277c8f9",
  "embedding_space_id": "66b3b4906b2d29b4e5b80b57",
  "model_id": null,
  "upload_ids": null,
  "finished": false,
  "error": false,
  "message": null,
  "error_msg": null,
  "error_details": null,
  "error_time": null,
  "exception": null,
  "warnings": [],
  "start_time": null,
  "end_time": null,
  "celery_id": "355a2cbb-9e74-4a9b-98f9-8bc01688010d",
  "changed_ids": null,
  "finished_stats": null,
  "request_args": {
    "job_type": "embedding-space-projection-create",
    "skip_auto_load": false,
    "project_id": "66b3b30dfcba35085277c8f9",
    "embedding_space_id": "66b3b4906b2d29b4e5b80b57"
  },
  "incremental_status": null,
  "last_updat

## From the docs on what to tell people to do 

In [15]:
import featrixclient as ft

# Split the data
#df_train, df_test = train_test_split(df, test_size=0.25)
 
# Connect to the Featrix service with your API key; create one at https://app.featrix.com/
featrix = ft.new_client(url=url)

# Here we assume that we have used the GUI to (1) create a demo project called 'DemoProject' and (2) uploaded a training csv file.
# If you need a train.csv file, we keep one handy at https://bits.featrix.com/demo-data/github.com-anujtiwari21/train.csv
demoProject = None
try:
    demoProject = featrix.get_project_by_name("DemoProject")
except:
    pass
if demoProject is None:
    demoProject = featrix.create_project("DemoProject")
    upload = featrix.upload_file("../data/animals-1k.csv")
    demoProject = demoProject.associate(upload)
# 

# Here we create a new embedding space (foundational model) and train it on the data that we have uploaded into 'demoProject'
embeddingSpace = demoProject.create_embedding_space(wait_for_completion=True)

# We can create multiple neural functions (predictive models) within an embedding space.
# This lets us re-use representations for different predictions without retraining the embedding space.
# Note, too, that you could train the model on a different training set than the embedding space, if you want to zero in on something
# for a specific model.
nf = embeddingSpace.create_neural_function(target_field='weight',
                                           wait_for_completion=True)

# Run predictions
#result = nf.predict(df_test)

# Now result is a list of classifications in the same symbols as the target column

Training Embedding Space:  Running: training;    100.0% complete (00:00:00 remaining); epoch = 5/5; current loss = 1.108


TypeError: create_neural_function() missing 1 required positional argument: 'target_fields'

In [17]:
nf = embeddingSpace.create_neural_function(target_fields='weight',
                                           wait_for_completion=True)

Training Neural Function predict_weight:  Running: training;    100.0% complete (00:00:00 remaining); epoch = 76/75; current loss = 0.437, validation loss: 0.11660481244325638


In [12]:
demoProject = featrix.get_project_by_name("DemoProject")
print(demoProject) 

None
